In [ ]:
%tensorflow_version 1.x
import os
import h5py
import random
import datetime
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import keras
from keras.optimizers import Adam
from keras.applications import VGG19
from keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import TensorBoard
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation, Flatten, Dropout, Input, concatenate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/')

Mounted at /content/drive


# Read the dataset and preprocess

In [ ]:
# f =  h5py.File('My Drive/Stethoscope/13A_data.hdf5', "r+")

# specs = np.array(f["specs"]).astype("float32")
# sonotypes = np.array(f["sonotypes"])
# times = np.array(f["times"]).astype("float32")
# freqs = np.array(f["freqs"]).astype("float32")

# f.close()

## By groups

In [ ]:
f =  h5py.File('My Drive/Stethoscope/data_group.hdf5', "r+")

specs_h5 = np.array(f["specs"]).astype("float32")
sonotypes_h5 = np.array(f["groups"])
times_h5 = np.array(f["times"]).astype("float32")
freqs_h5 = np.array(f["freqs"]).astype("float32")

f.close()

In [ ]:
# append x_times an x_freqs to be auxiliary_input
aux_input_h5 = np.append(times_h5,freqs_h5, axis = 1)


print(aux_input_h5.shape)
print(aux_input_h5[10])

(2881, 4)
[22293.    22321.469  7993.379  8626.394]


In [ ]:
# use only "b" and "i"
index_b = np.argwhere(sonotypes_h5 == b'b').flatten()
index_i = np.argwhere(sonotypes_h5 == b'i').flatten()

print("number of data in group i: %i" % len(index_i))

# randomly choose index in b 
# to ensure we have same number of type b and i
random.shuffle(index_b)
index_b_resized = index_b[:len(index_i)]
print("number of data in group b after resizing: %i" % len(index_b_resized))

# get the data of b and i
specs_b = specs_h5[index_b_resized]
specs_i = specs_h5[index_i]
aux_input_b = aux_input_h5[index_b_resized]
aux_input_i = aux_input_h5[index_i]

# print(index_b_resized[1])
# print(times_b[1])
# print(times[index_b_resized[1]])

# put them together
specs = np.append(specs_b, specs_i, axis = 0)
aux_input = np.append(aux_input_b, aux_input_i, axis= 0)
sonotypes = np.append(np.repeat(b'b', len(index_i)), np.repeat(b'i', len(index_i)))

print(specs.shape)
print(aux_input.shape)
print(sonotypes)

number of data in group i: 1021
number of data in group b after resizing: 1021
(2042, 224, 224, 3)
(2042, 4)
[b'b' b'b' b'b' ... b'i' b'i' b'i']


## By top k


In [ ]:
f =  h5py.File('My Drive/Stethoscope/augment_data_0922.hdf5', "r")

specs_h5 = np.array(f["specs"]).astype("float32")
sonotypes_h5 = np.array(f["sonotypes"]).astype("float32")
times_h5 = np.array(f["times"]).astype("float32")
freqs_h5 = np.array(f["freqs"]).astype("float32")
# groups_h5 = np.array(f["groups"])
# selection_h5 = np.array(f["selections"])

f.close()

In [ ]:
# append x_times an x_freqs to be auxiliary_input
aux_input_h5 = np.append(times_h5,freqs_h5, axis = 1)

print(aux_input_h5.shape)
print(aux_input_h5[10])

(2999, 4)
[22293.834 22337.82   7993.379  8626.394]


In [ ]:
# create the dictionary for sonotypes and groups
sono2group = dict(zip(sonotypes_h5,groups_h5))

# get the data for top k sonotypes
s_unique, s_freq = np.unique(sonotypes_h5,return_counts=True)
s_freq_order = np.argsort(s_freq)[::-1]
s_freq_desc = s_freq[s_freq_order]

print(len(s_unique))
print(s_unique[s_freq_order][:10])
print(s_freq_desc[:10])

441
[ 52. 138.  25. 463. 236.   1. 167.  86. 175. 220.]
[175 119 114  91  74  69  60  57  49  47]


##### use a balanced set of input


In [ ]:
numUsed =7
groupUsed = b'b' # use bird for now
typeUsed = []
min_num = 1000


# typeUsed=[52,138,463]
# min_num =91
index = 0
while len(typeUsed) < numUsed:
  cur_type = s_unique[s_freq_order][index]
  if sono2group[cur_type] == groupUsed:
    typeUsed.append(cur_type)
    if s_freq_desc[index] < min_num:
      min_num = s_freq_desc[index]
  index += 1

print(typeUsed)
print(min_num)
# typeUsed = s_unique[s_freq_order][:numUsed]
# min_num = s_freq_desc[numUsed -1] # the number of every sonoytpes

specs = []
aux_input = []
sonotypes = []
selections= []

for i in range(len(typeUsed)):
  # get index of the current type of spec
  cur_index = np.argwhere(sonotypes_h5 == typeUsed[i]).flatten()
  # randomly choose index in b 
  # to ensure we have same number of type b and i
  random.shuffle(cur_index)
  cur_index_resized = cur_index[:min_num]
  if len(specs):
    specs = np.append(specs, specs_h5[cur_index_resized], axis = 0)
  else:
    specs = specs_h5[cur_index_resized]

  if len(aux_input):
    aux_input = np.append(aux_input, aux_input_h5[cur_index_resized], axis= 0)
  else:
    aux_input = aux_input_h5[cur_index_resized]
  
  if len(sonotypes):
    sonotypes = np.append(sonotypes, np.repeat(i, min_num))
  else:
    sonotypes = np.repeat(i, min_num)

  if len(selections):
    selections = np.append(selections, selection_h5[cur_index_resized], axis= 0)
  else:
    selections = selection_h5[cur_index_resized]

print(specs.shape)
print(aux_input.shape)
# print(selections)
# print(sonotypes)
specs_keep = np.copy(specs)

[52, 138, 463]
91
(273, 224, 224, 3)
(273, 4)


#### use a inbalanced set of input

In [ ]:
numUsed = 4
typeUsed = s_unique[s_freq_order][:numUsed]
# min_num = s_freq_desc[numUsed -1] # the number of every sonoytpes

specs = []
aux_input = []
sonotypes = []

for i in range(numUsed):
  # get index of the current type of spec
  cur_index = np.argwhere(sonotypes_h5 == typeUsed[i]).flatten()
  # randomly choose index in b 
  # to ensure we have same number of type b and i
  # random.shuffle(cur_index)
  # cur_index_resized = cur_index[:min_num]
  if len(specs):
    specs = np.append(specs, specs_h5[cur_index], axis = 0)
  else:
    specs = specs_h5[cur_index]

  if len(aux_input):
    aux_input = np.append(aux_input, aux_input_h5[cur_index], axis= 0)
  else:
    aux_input = aux_input_h5[cur_index]
  
  if len(sonotypes):
    sonotypes = np.append(sonotypes, np.repeat(i, len(cur_index)))
  else:
    sonotypes = np.repeat(i,len(cur_index))

print(specs.shape)
print(aux_input.shape)
# print(sonotypes)

(499, 224, 224, 3)
(499, 4)


### thresholding/Normalization

In [ ]:
#  use threshold to get rid of small data
for i in range(len(specs)):
  threshold = np.percentile(specs[i], 80)
  specs[i] = np.where(specs[i] < threshold, 0, 255)

In [ ]:
# normalization、 the data
for i in range(len(specs)):
  cur_spec = specs_keep[i]
  s_min = np.amin(cur_spec)
  s_max = np.amax(cur_spec)
  # specs[i] = (cur_spec - s_min)/(s_max - s_min) * 255
  specs[i] =(cur_spec - s_min)/(s_max - s_min) 

### randomly delete one data

In [ ]:
# randomly delete one less data from current test data
num_sono = 3
cur_specs = np.copy(specs)
cur_aux = np.copy(aux_input)
cur_sonotypes = np.copy(sonotypes)
cur_selections = np.copy(selections)

data_per_sono = int(cur_specs.shape[0] / num_sono)
for i in range(num_sono):
  index = random.randint(0, data_per_sono - 1)  + data_per_sono * i - i
  print(index)
  cur_specs = np.delete(cur_specs, index, 0)
  cur_aux = np.delete(cur_aux,index,0)
  cur_sonotypes = np.delete(cur_sonotypes,index,0)
  cur_selections = np.delete(cur_selections,index,0)

print(data_per_sono)
print(cur_specs.shape)
print(cur_aux.shape)

1
12
22
12
(33, 224, 224, 3)
(33, 4)


In [ ]:
specs = cur_specs
aux_input = cur_aux
sonotypes = cur_sonotypes
selections = cur_selections

## Seperate test and train
Problem here: Althuogh sklearn train_test_split split randomly, in our case when we have the small dataset, chances are that the test/validation for differenct sonotypes differ a lot.
fixed in the later version of classification_with_augmentation.

In [ ]:
# seperate the data into training, validation, and testing
# 0.8 for train, 0.1 for val and test
x_train, x_test_val, y_train, y_test_val, aux_train, aux_test_val, sel_train, sel_test_val = train_test_split(specs, sonotypes, aux_input, selections, test_size=0.2)

x_val, x_test, y_val, y_test, aux_val, aux_test, sel_val, sel_test = train_test_split(x_test_val, y_test_val, aux_test_val, sel_test_val, test_size=0.5)

x_test = [x_test, aux_test]
cat_y_test = to_categorical(pd.factorize(y_test)[0],num_classes= len(typeUsed))

# # test: work without aux input
# aux_train = np.zeros((len(x_train), 4))
# aux_test = np.zeros((len(x_test),4))

print(len(x_train))
print(len(x_test))

245
28


# Test the model with MINST dataset

In [ ]:
from keras.datasets.mnist import load_data
import cv2

types = [1,5,4,3,2]
num = [72,72,72,72,72]
# load the data - it returns 2 tuples of digits & labels - one for
# the train set & the other for the test set
(train_digits, train_labels), (test_digits, test_labels) = load_data()

train_num = np.multiply(num, 0.9)
test_num = np.multiply(num, 0.1)

# re-shape the images data
train_data = []
test_data = []
train_y = []
test_y = []
for j in range(len(types)):
  cur_train = []
  for i in range(len(train_digits)):
    if train_labels[i] == types[j]:
      spec_resized=cv2.resize(train_digits[i],(224,224))
      spec_resized=cv2.cvtColor(spec_resized.astype('float32'), cv2.COLOR_BGR2RGB) #cv2 does not accept float64 
      spec_resized = np.flip(spec_resized,0)
      cur_train.append(spec_resized)
      train_y.append(train_labels[i])
    if len(cur_train) >= train_num[j]:
      if len(train_data):
        train_data = np.append(train_data, cur_train, axis = 0)
      else:
        train_data = cur_train
      break
  

  cur_test = []
  for i in range(len(test_digits)):
    if test_labels[i] == types[j]:
      spec_resized=cv2.resize(test_digits[i],(224,224))
      spec_resized=cv2.cvtColor(spec_resized.astype('float32'), cv2.COLOR_BGR2RGB) #cv2 does not accept float64 
      spec_resized = np.flip(spec_resized,0)
      cur_test.append(spec_resized)
      test_y.append(test_labels[i])
    if len(cur_test) >= test_num[j]:
      if len(test_data):
        test_data = np.append(test_data, cur_test, axis = 0)
      else:
        test_data = cur_test
      break

train_digits = []
train_labels = []
test_digits = []
test_labels = []

11493376/11490434 [==============================] - 1s 0us/step


In [ ]:
len(train_data), len(test_data)

(325, 40)

In [ ]:
from keras.utils import to_categorical

cat_y_train = to_categorical(train_y[:1600])
cat_y_test = to_categorical(test_y[:200])
aux_train = np.zeros((len(train_data), 4))
aux_test = np.zeros((len(test_data),4))
x_train = train_data
x_test = test_data

len(aux_train), len(aux_test)

(325, 40)

# To Categorical

In [ ]:
# Convert labels to a categorical array

num_c = max(len(np.unique(y_val)), len(np.unique(y_train)))
print(num_c)
cat_y_val = to_categorical(pd.factorize(y_val)[0],num_classes= num_c)
cat_y_train = to_categorical(pd.factorize(y_train)[0], num_classes= num_c)

3


# Model


In [ ]:
config = dict(
    dropout = 0.55,
    hidden = 1024,
    learn_rate = 0.00001,
    epochs = 30,
    )

In [ ]:
def build_finetune_model(base_model, dropouts, fc_layers, num_classes):
    for layer in base_model.layers:
       layer.trainable = False

    x = base_model.output
    x = Flatten()(x)

    # add input layer
    auxiliary_input = Input(shape=(4,), name='aux_input')
    x = concatenate([x, auxiliary_input])

    for fc, drop in zip(fc_layers, dropouts):
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(drop)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=[base_model.input,auxiliary_input], outputs=predictions)

    return finetune_model

In [ ]:
model = None
keras.backend.clear_session()
model = VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
model = build_finetune_model(model, 
                             [config["dropout"], config["dropout"]], 
                             [config["hidden"], config["hidden"]], 
                             cat_y_train.shape[1])

In [ ]:
model.summary()

In [ ]:
plot_model(model,to_file = 'My Drive/Stethoscope/model.png')

# Training

In [ ]:
filepath = 'My Drive/Stethoscope/model_group.hdf5'

earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

opt = Adam(lr=config["learn_rate"])
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit(x=[x_train, aux_train], y=cat_y_train, validation_data=([x_val, aux_val], cat_y_val), epochs=50, verbose = 2)
history = model.fit(x=[x_train, aux_train], y=cat_y_train, validation_data=([x_val, aux_val], cat_y_val), epochs=config["epochs"], callbacks=[earlystop, checkpoint])

In [ ]:
# load the model with best loss
model = None
keras.backend.clear_session()
model = load_model (filepath)
# evaluation
results = model.evaluate( x= x_test, y=cat_y_test)
print("best loss test loss, test acc:", results)